# Подключение

In [1]:
import pyodbc

In [51]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.199.13.60;DATABASE=rway;UID=vkomarnitskii;PWD=Rway1')
cursor = conn.cursor()


## Pandas

In [3]:
import pandas as pd
from math import isnan

Тут можно установить параметры отображения для pandas. Сколько показывать строк и столбцов соответственно.

In [4]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 150

### Отладка


In [14]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции
    """
    import time
    
    def wrapper(*args, **kwargs):
        t = time.process_time()
        res = func(*args, **kwargs)
        print(f'{func.__name__} выполнилась за время {time.process_time() - t}s')
        return res
    
    return wrapper

## Необходимые функции

Позволяет получить список характеристик для всех предложений, указанных в list_of_keys. Предложения указываются в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

Преобразование из байтов в такую строку делается следующим образом:
```python
# type(x) == bytes
result = '0x' + x.hex().upper()
```

In [53]:
@benchmark
def get_harks_by_object_keys(task_id):
    """
    :param task_id: id задачи
    :type task_id: str
    :return: DataFrame, в котором указаны характеристики и их значения для всех объектов из задачи task_id
    """
    
#     in_expr = f'({", ".join(list_of_keys)})'
    query = '''
    SELECT
    t_har.Наименование,
    CASE 
        WHEN t.Значение_Тип = 04 THEN CAST(t.Значение_Дата AS varchar)
        WHEN t.Значение_Тип = 03 THEN CAST(t.Значение_Число AS varchar)
        WHEN t.Значение_Тип = 05 THEN t.Значение_Строка
        WHEN t.Значение_Тип = 01 THEN 
        CASE
            WHEN t.Значение = 0x869CEA7372E5CA3E468F18025D610C0B THEN 'Да'
            WHEN t.Значение = 0xB0CD188371B9A88E431B073454A9489F THEN 'Нет'
            ELSE 'НетДанных'
        END
        WHEN t.Значение_Тип = 08 THEN 
        COALESCE(
            (SELECT Наименование FROM [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop WHERE t_dop.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types WHERE t_types.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыСделки] t_types_1 WHERE t_types_1.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ЛогическиеЗначения] t_logic WHERE t_logic.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыВерифицированности] t_verif WHERE t_verif.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ПодСегменты] t_podseg WHERE t_podseg.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[Застройщики] t_zastr WHERE t_zastr.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations WHERE t_stations.Ссылка = t.Значение)                  
        )
    END
        AS Значение,
    t.Объект AS Ссылка
FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
    INNER JOIN [rway].[ПВХ].[Характеристики] t_har
        ON t_har.[Ссылка] = t.[Характеристика]
    INNER JOIN [rway].[РегистрСведений].[ПредложенияЗадач] reg_task
        ON reg_task.Предложение = t.Объект
        AND reg_task.Задача in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{}')
    '''.format(task_id)
    
    cursor.execute(query)
#     data_df = pd.read_sql(sql=query, con=conn)

    # Преобразование данных, которые вернул запрос в список списков, одновременно все байты преобразуются в hex
    data = list(map(lambda x: list(x), cursor.fetchall()))
    data_df = pd.DataFrame(data, columns=list(map(lambda x: x[0], cursor.description)))
    # Создание словаря для DataFrame
    return data_df.pivot(index='Ссылка', columns='Наименование', values='Значение')

    

Функция позволяет получить все предложения по конкретной задаче. Задача указывается в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

In [49]:
@benchmark
def get_base_info_by_task_id(task_id, offers_count=None):
    """
    :param task_id: id задания
    :param offers_count: Количество предложений для выбора из базы (пустое для выбора всех предложений)
    :type task_id: str
    :return: DataFrame, содержащий все поля, которые можно получить из таблицы "ПредложенияЗадач"
    """
    
    top = '' if not offers_count else 'TOP {}'.format(offers_count)
    query = '''
    SELECT {}
          t.Код,
          t.Ссылка,
          t_source.Наименование AS Источник,
          t.АдресAhunter,
          t.АктуальнаяСсылкаИсточника,
          t.ДатаПересмотраЭкспозиции,
          t.ДатаПроверкиАктуальности,
          t.ДатаРазмещения,
          t.Город,
          t.Описание,
          (SELECT Наименование FROM [rway].[Справочник].[Подсегменты] WHERE Ссылка = t.Подсегмент) AS Подсегмент,
          (SELECT Наименование FROM [rway].[Справочник].[Сегменты] WHERE Ссылка = t.Сегмент) AS Сегмент,
          (SELECT Наименование FROM [rway].[Справочник].[СубъектыРФ] WHERE Ссылка = t.Субъект) AS Субъект,
          t.СсылкаИсточника,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыРынка] WHERE Ссылка = t.ТипРынка) AS ТипРынка,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыСделки] WHERE Ссылка = t.ТипСделки) AS ТипСделки
    FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
        JOIN [rway].[РегистрСведений].[ПредложенияЗадач] t_offer
            ON Ссылка = t_offer.[Предложение]
        JOIN [rway].[dbo].[_Task62] t_task
            ON t_task.[_IDRRef] = t_offer.[Задача] AND t_task.[_IDRRef] in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{}')
        JOIN [rway].[Справочник].[Источники] t_source
            ON t_source.Ссылка = ИсточникИнформации

    '''.format(top, task_id)
    
    cursor.execute(query)    
    data = cursor.fetchall()
    # Те же самые преобрзования, как в функции с характеристиками
    df = pd.DataFrame(map(lambda x: list(x), data))
    df.columns = list(map(lambda x: x[0], cursor.description))
#     df = pd.read_sql(sql=query, con=conn)
    return df
    

# Боевой запуск

In [52]:
# Сбор основных данных из "ПредложенияОбъектовНедвижимости"
base_df = get_base_info_by_task_id('0001-0405')

get_base_info_by_task_id выполнилась за время 15.21875s


In [54]:
base_df



,Код,Ссылка,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки
0,190725W00628329,b'\x97\x18r\x92\xd3\x90\xf1\rB\x0f)?F\xb0\x12\...,MOVE.ru,"Респ Крым, г Ялта, ул Пушкинская, дом 26",,2019-07-25 00:00:00,2019-07-25 17:49:57,2019-07-18 00:00:00,Ялта,"Продaётся тоpгoвое помещениe в центpе Ялты, у...",торговый,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_kommer...,None,None
1,190725W00083177,b'\x80J\xc9^W\xa5-PE\x89\xed\xe7\xc8J\xa5\x0f',КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:27:01,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None
2,190725W00118461,b'\x80p\xc4^)\x1bKDKJ\xcb\x98\xa9l\xd6D',КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 15:01:32,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None
3,190725W00143303,b'\x81J\xa7\x9d-\x92\x9b\x8bL\xa3D_#\x9b|\x8c',КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 15:30:38,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None
4,190725W00088154,b'\x82\x1b\x8dExG|pBB1\xf6\xc6~\xfes',КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:31:52,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None
5,190725W00113236,b'\x82<x\xf5\xb0\xa2x\xd6N\xfd#b\xe1\xa2\xfa%',КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:57:49,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None
6,190725W00083083,b'\x82N\x91{\xf0\xaf\x10\xf7M\xa4\xe7\xcb\xc5n...,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:28:08,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None
7,190725W00118615,b'\x82_\xbc(\xa2\x98_\xeaM\x93\x91\xcbs\xf9fR',КУПИ.ру,,,2019-07-25 00:00:00,2019-07-25 15:02:45,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Москва г,https://moskva.qp.ru/?standart=true,Вторичный,None
8,190725W00113047,"b""\x82s\x9b\x89_'\xae\xd0M\xccg\xb2K4f[""",КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:58:34,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None
9,190725W00113048,b'\x82\xd2\x8b)>f\xc4\x0fF\x8f\xe7\xdcY\x9f\xd9!',КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:58:45,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None


In [55]:
# Сбор характеристик по ссылкам из основного DataFrame base_df
har_df = get_harks_by_object_keys('0001-0405')

get_harks_by_object_keys выполнилась за время 96.03125s


In [56]:
har_df

Наименование,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Водоснабжение,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Интернет,Класс Объекта,Количество Комнат,Количество Этажей,Кондиционирование,Лифты,Материал Наружных Стен,Минут До Ближайшей Станции Метро,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,Охрана Предложения,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расположение Относительно Первой Линии Домов,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Объекта,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Электричество,Этаж
Ссылка,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b'\x80\x00\x01#\xc4QYuC\x82\xf6\xb5\xe4kfm',NaN,NaN,Рябикова ул,дом № 75,NaN,NaN,NaN,NaN,48.289042,54.272843,NaN,2019-07-22 00:00:00,Сдаю свободного назначения помещение 190.0 м² ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,торговая площадь,NaN,190.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Костя,кв.м.,руб./мес.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(951) 0987299,NaN,NaN,NaN,NaN,NaN,25000.000000,131.578947,NaN,1
b'\x80\x00\x02\x9c|\xb3%\x9cG\x8fmJ\x8f\xc9\xef\xd8',NaN,NaN,Космонавтов пр-кт,дом № 32В/21В,NaN,NaN,NaN,NaN,39.713863,47.296090,NaN,2019-07-25 15:57:00,"Аренда коммерческой недвижимости, 35 м2, Росто...",NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,торговая площадь,NaN,35.000000,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Продавец,кв.м.,руб./мес.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(928) 2266935,NaN,NaN,NaN,None,NaN,26000.000000,742.857143,NaN,2
b'\x80\x00\x04\x8a=+\xc1pHh)\xdc\xbbF\x9f\xe5',NaN,NaN,Рыбинская 3-я ул,"дом № 18, строение 1",NaN,NaN,NaN,NaN,37.659127,55.789683,NaN,2019-07-22 00:00:00,"Аренда коммерческой недвижимости, 286м 2, Моск...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,офис,NaN,286.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Продавец,кв.м.,руб.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(926) 9175628,NaN,NaN,NaN,NaN,NaN,572000.000000,2000.000000,NaN,"6, 2, 5"
b'\x80\x00\x05\x02\xb5\xa2\xcf\x86MG\x7fJ\xe70M&',NaN,NaN,им Бабушкина ул,,NaN,None,NaN,NaN,38.955820,45.049704,0.000000,2019-07-25 20:40:12,"Офис, 1600 м2, им Бабушкина,-",None,None,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,офис,Чистовая,1600.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Тарасова Евгения, Агентство",кв.м.,руб./мес.,NaN,0.000000,NaN,Без Проведения Торгов,NaN,NaN,(967) 9300021,NaN,бизнес-центр,NaN,бизнес-центр,NaN,183700.000000,114.812500,NaN,5
"b""\x80\x00\x06~\xb0'\xa1\x06B\xf1\x81\x91\x84Fe\x04""",NaN,NaN,Мира пр-кт,дом № 40,NaN,NaN,NaN,NaN,37.633949,55.780205,NaN,2019-07-25 21:54:35,"Помещение свободного назначения м. Пр.Мира, 27...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,офис,NaN,276.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Городской Департамент Недвижимости,кв.м.,руб./мес.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(958) 7696931,NaN,NaN,NaN,NaN,NaN,720000.000000,2608.695652,NaN,9
b'\x80\x00\x14\xcb3\x0f\xa3QO\x01\x11r\x19\xd4\x8f\x16',NaN,NaN,Ленина пр-кт,дом № 112Б,NaN,None,NaN,0.000000,37.581900,54.154391,0.000000,2019-07-26 11:46:39,"Свободное назначение, 74 м2",NaN,НетДанных,NaN,9.000000,None,NaN,None,NaN,0.000000,Включая,свободное,None,74.000000,,Отдельный Вход,NaN,0.000000,None,74.000000,0.000000,None,NaN,NaN,Сергей Парфенов,кв.м.,руб./мес.,NaN,0.000000,None,Без Проведения Торгов,Действующий,NaN,(906) 5310114,None,NaN,None,жилой дом,,58978.000000,797.000000,NaN,
b'\x80\x00\x15\xf3\xc32#n@\xb6!F_V\xcd\xa6',NaN,NaN,Кирова пр-кт,дом № 143,NaN,None,NaN,NaN,NaN,NaN,0.000000,2019-07-25 14:13:16,"Помещение свободного назн

In [88]:
# Собираем обе таблицы в одну большую
result_df = pd.merge(base_df, har_df, on='Ссылка', how='left')

In [91]:
nan = float('nan')
s = result_df.replace('', nan).count()

In [99]:
for k, v in s.items():
    print(k, v)

Код 862512
Ссылка 862513
Источник 862513
АдресAhunter 855312
АктуальнаяСсылкаИсточника 80173
ДатаПересмотраЭкспозиции 862513
ДатаПроверкиАктуальности 862513
ДатаРазмещения 862451
Город 819494
Описание 862505
Подсегмент 810583
Сегмент 862513
Субъект 862479
СсылкаИсточника 862513
ТипРынка 861820
ТипСделки 860329
3.Муниципальное Образование 5934
5.Внутригородской Район 4867
7.Улица 799693
8.Дом 643898
Адрес (временная) 67324
Ближайшая Станция Метро 122
Водоснабжение 116
Высота Потолков 279103
ГеоКод Долгота 799993
ГеоКод Широта 799993
Год Постройки 456489
Дата Сбора Информации 862513
Заголовок 848959
Интернет 42557
Класс Объекта 162884
Количество Комнат 760
Количество Этажей 721605
Кондиционирование 0
Лифты 23669
Материал Наружных Стен 20645
Минут До Ближайшей Станции Метро 23669
Мощность Электричества 101407
НДС 74684
Назначение Объекта Предложения 812229
Наличие Отделки Помещений 37176
Общая Площадь Предложения 862467
Ориентир 35873
Отдельный Вход 39799
Охрана Предложения 24570
Парковка

In [86]:
result_df[result_df['Источник'] == 'Издательский дом Ярмарка'].replace('', float('nan'))


,Код,Ссылка,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Водоснабжение,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Интернет,Класс Объекта,Количество Комнат,Количество Этажей,Кондиционирование,Лифты,Материал Наружных Стен,Минут До Ближайшей Станции Метро,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,Охрана Предложения,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расположение Относительно Первой Линии Домов,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Объекта,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Электричество,Этаж
118085,190725W00079970,"b'\x96(c\xda\xcbS\xfe\xc4N\x97\xe3\xdb[z,\x90'",Издательский дом Ярмарка,"обл Калининградская, г Калининград, ул Барнаул...",NaN,2019-07-25 00:00:00,2019-07-25 14:19:51,2019-07-23 00:00:00,Калининград,родается здание довоенной постройки в центре г...,ПСН,Коммерческая Недвижимость,Калининградская обл,https://ya39.ru/realty/commercial/prodat/zdani...,Вторичный,Продажа,NaN,NaN,Барнаульская ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:19:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,свободное,NaN,960.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Александра,кв.м.,руб.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(401) 2400621,NaN,NaN,NaN,ОСЗ,NaN,92000000.000000,95833.333333,NaN,NaN
119715,190725W00079972,b'\x81\xb9\x05\xcc\x19\x10\xe1\x06F3\xfe\x02w\...,Издательский дом Ярмарка,"обл Калининградская, р-н Гурьевский, п Шоссейн...",NaN,2019-07-25 00:00:00,2019-07-25 14:19:59,2019-07-23 00:00:00,NaN,"Продаётся здание под СТО с участком, всё в с...",сфера услуг,Коммерческая Недвижимость,Калининградская обл,https://ya39.ru/realty/commercial/prodat/avtos...,Вторичный,Продажа,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:19:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,станция техобслуживания,NaN,296.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Продавец,кв.м.,руб.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(921) 7105805,NaN,NaN,NaN,None,NaN,6499000.000000,21956.081081,NaN,NaN
299283,190725W00074690,b'\xad\xeb\xd5\xb02\xcb*<L\x02\xa8\xcaZ=\xa66',Издательский дом Ярмарка,"обл Калининградская, г Калининград, ул Литовск...",NaN,2019-07-25 00:00:00,2019-07-25 14:17:58,2019-07-23 00:00:00,Калининград,Удобный отличный офис в высоком цоколе многоэт...,ПСН,Коммерческая Недвижимость,Калининградская обл,https://ya39.ru/realty/commercial/prodat/pomes...,Вторичный,Продажа,NaN,NaN,Литовский Вал ул,дом № 87Б,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:17:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,свободное,NaN,41.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Елена,кв.м.,руб.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(981) 4697611,NaN,NaN,NaN,None,NaN,2170000.000000,52926.829268,NaN,NaN
403599,190725W00079971,"b'\xb3\xa4\x96\xc3""%\xc1\xa3M`\x99\xde\x19\x95...",Издательский дом Ярмарка,"обл Калининградская, г Калининград, ул Ю.Гагар...",NaN,2019-07-25 00:00:00,2019-07-25 14:22:58,2019-07-19 00:00:00,Калининград,Сдаем в аренду производственные помещения от 1...,производственно-складской,Коммерческая Недвижимость,Калининградская обл,https://ya39.ru/realty/commercial/sdat/zdanie_...,Вторичный,Аренда,NaN,NaN,Ю.Гагарина ул,дом № 225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:22:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,производство,NaN,120.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ооо \\\\\\

In [104]:
def make_test():
    new_df = pd.DataFrame(columns=set(result_df.loc[:, 'Источник']), index=list(result_df.columns.fillna(0)) + ['Всего'])
    nan = float('nan')
    for source in set(result_df['Источник']):
        filtered = result_df[result_df['Источник'] == source]
        new_df.loc['Всего', source] = len(filtered)
        count_s = filtered.replace('', nan).replace('НетДанных', nan).replace('0.000000', nan).count()
        for k, v in count_s.items():
            new_df.loc[k, source] = v * 100 / new_df.loc['Всего', source]
    return new_df

Функция, в которой и происходит тестирование.

In [105]:
test = make_test()

Выгрузка данных в `csv` файл для удобного просмотра в Excel

In [106]:
test.to_csv('test_result.csv', header=list(test.columns), encoding='cp1251', sep=';')
test

,Необходимая недвижимость,Buy business portal,Адвекс - АН,Издательский дом Ярмарка,ИС Центр,AVITO.ru,Zdanie.info,Granta24,Олимп - АН,ЦИАН,Градстрой - АН,Arendator.ru,TheProperty.ru,Multilisting.Su,Золотая Середина - АН,INFOLINE,Sibdom.Ru,Связист - АН,Life-Realty.ru,Rosnedv.ru,Tomsk.Ru09,Kalugahouse.ru,Уральская палата недвижимости,Vladimirhouse.ru,RAUI,Доска.ру,FARPOST.ru,БИБОСС,Аякс-Риэлт,Youla.Ru,Rosrealt.ru,Market.sakh.com,КУПИ.ру,VSE42.RU,ГдеЭтотДом.ру,RealtyMag.RU,Вестум.ru,Из рук в руки,Брянский сервер недвижимости,Элиант Недвижимость - АН,MLSN.ru,MOVE.ru,RealtyVision.ru,Квадрат.ру,Домовой45,IRK.ru,Ryazanhouse.ru,N1.ru,Tulahouse.ru,Tvoyadres.ru,Яндекс Недвижимость,Gde.ru,Бюллетень Недвижимости,СИТИСТАР,MIRKVARTIR.ru
Код,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,99.9996,100,100,100,100,100,100,100,100,100,100,100,100,100
Ссылка,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
Источник,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
АдресAhunter,99.297,98.6425,100,100,98.4343,99.9329,99.9865,100,0,99.9562,100,99.815,99.6827,99.8523,100,100,98.8496,99.3865,96.0573,99.9034,100,92.3741,99.7661,29.0323,99.8521,92,95.0288,99.8868,99.9417,95.8287,95.0964,100,99.1169,100,99.7972,98.0268,99.9748,98.1081,100,96.9529,99.4512,99.7463,98.9028,99.8,100,86.6667,0,99.7621,6.45161,99.6899,99.7313,97.3958,99.8199,99.1489,98.8429
АктуальнаяСсылкаИсточника,0,100,0,0,0,0,0,0,0,0,0,100,0,100,0,0,0,0,0,99.0657,100,0,0,0,0,100,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,100,0,0,0,100
ДатаПересмотраЭкспозиции,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
ДатаПроверкиАктуальности,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
ДатаРазмещения,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,97.7666,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
Город,91.2127,86.2745,100,75,90.776,94.243,99.314,99.2,0,97.0952,100,98.8661,98.8176,92.444,100,86.5,94.9055,93.5583,92.4731,93.75,98.5531,92.0863,96.3158,25.8065,96.2229,84,90.8141,99.5285,98.6597,91.3366,90.456,97.9695,90.9289,98.8864,95.8345,95.2759,99.6471,94.4216,83.3333,96.1219,98.0241,94.5867,85.5799,98,50,86.6667,0,98.3022,6.45161,98.062,94.6507,88.9323,98.5848,98.5816,82.8284
Описание,100,100,100,100,100,99.9985,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,99.9931,100,100,100,100,99.8902,99.9996,100,100,100,100,100,100,100,100,100,100,100,100,100


## Для тестов


In [ ]:
cursor = conn.cursor()
query = '''
/****** Скрипт для команды SelectTopNRows из среды SSMS  ******/
SELECT TOP 1000
    *
FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости]
    
'''
# WHERE _FLD198 = '0001-0402-0001'

cursor.execute(query)
data = cursor.fetchall()
df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
df.columns = list(map(lambda x: x[0], cursor.description))
df


In [ ]:
query = '''
/****** Скрипт для команды SelectTopNRows из среды SSMS  ******/
SELECT TOP 1000
    t_source.Наименование
FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
    JOIN [rway].[Справочник].[Источники] t_source
        ON t_source.Ссылка = t.ИсточникИнформации
    
'''
# WHERE _FLD198 = '0001-0402-0001'

df = pd.read_sql(sql=query, con=conn)
df